In [ ]:
#Install Python library for using SQL on Jupyter Notebook
!pip install ipython-sql

In [ ]:
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [ ]:
#load sql module for running sql queries
%load_ext sql

In [ ]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt


In [ ]:
file_path="CovidDeaths.xlsx"
db_name="db_sqlite"
table_name="CovidDeaths"

In [ ]:
#clone repository database



In [ ]:
#Read the  file
df = pd.read_excel(file_path,nrows=10000)

In [ ]:
%sql sqlite:///db_sqlite.db

In [ ]:
#create a connection to the db
conn =sqlite3.connect('db_sqlite.db')

In [ ]:
#converts the DataFrame into a sql Table
df.to_sql(table_name,conn, if_exists='replace', index=False)

In [ ]:
from typing import List

def unique(nom_table, col):
    """
    Implémentation simple de la fonction unique de pandas
    """
    query = f"select distinct {col} from {nom_table}"
    result = %sql {query}
    return result

def join(nom_table, join_table, on):
    """
    Implémentation simple de la fonction join de pandas
    """
    query = f"select * from {nom_table} join {join_table} on {on}"
    result = %sql {query}
    return result

def group_by(nom_table, by: List, limit=10):
    """
    Implémentation simple de la fonction group_by de pandas
    """
    query = f"select * from {nom_table} group by {','.join(by)} limit {limit}"
    result = %sql {query}
    return result

def simple_iloc(nom_table, indexes: List, limit=100):
    """
    Implémentation simple de la fonction iloc de pandas
    Renvoie les lignes de la table
    """
    idx = [str(id) for id in indexes]
    query = f"select * from {nom_table} where rowid in ({','.join(idx)}) LIMIT {limit}"
    result = %sql {query}
    return result

def simple_loc(nom_table, indexes: List, limit=100):
    """
    Implémentation simple de la fonction loc de pandas
    Renvoie les lignes d'index spécifiées de la table
    """
    query = f"select * from {nom_table} where index in ({','.join(indexes)}) LIMIT {limit}"
    result = %sql {query}
    return result

def rename_table(nom_table, new_name):
    """
    Renommer une table
    """
    query = f"ALTER TABLE {nom_table} RENAME TO {new_name}"
    result = %sql {query}
    return result

def get_columns(columns: List, nom_table):
    """
    Récupérer les colonnes spécifiées d'une table
    """
    query = f"select {','.join(columns)} from {nom_table}"
    result = %sql {query}
    return result

def shape_sql(nom_table):
    """
    Retourne la forme de la table (lignes, colonnes)
    """
    query_column = f"select count(*) from pragma_table_info('{nom_table}')"
    ncolumns = %sql {query_column}
    nrows = %sql select count(*) from {nom_table}
    col = int(ncolumns[0][0])
    rows = int(nrows[0][0])
    return (rows, col)

def head_sql(nom_table, n=5):
    """
    Fonction similaire à head() de pandas
    """
    query = %sql select * from {nom_table} LIMIT {n}
    return query

def tail_sql(nom_table, n=5):
    """
    Fonction similaire à tail() de pandas
    """
    query = %sql select * from {nom_table} ORDER BY {nom_table} DESC LIMIT {n}
    return query

def info_sql(nom_table):
    """
    Obtenir les informations sur la structure de la table
    """
    query = f"PRAGMA table_info({nom_table})"
    info = %sql {query}
    return info

def get_cols(nom_table):
    """
    Retourne les colonnes d'une table
    """
    query = f"select * from pragma_table_info('{nom_table}')"
    query = %sql {query}
    return query

def dropna(nom_table, listcol):
    """
    Supprimer les lignes contenant des valeurs manquantes (None) basées sur les colonnes spécifiées
    """
    colsdrop = f"{listcol[0]}"
    if len(listcol) > 1:
        colsdrop = "  <> 'None' AND ".join(listcol)  # Adapter à vos valeurs NA. Exemple : <colname> IS NOT NULL
    colsdrop += " <> 'None' "
    query = %sql select * from {nom_table} where {colsdrop} LIMIT 100
    return query

def describe(nom_table, colname):
    """
    Retourne le nombre, max, min, moyenne, écart-type et quelques quantiles
    """
    count = %sql select count(*) as count from {nom_table} where {colname} <> 'None'
    max = %sql select MAX({colname}) as max from {nom_table}
    min = %sql select MIN({colname}) as min from {nom_table}
    avg = %sql select AVG({colname}) as mean from {nom_table}

    q25 = quantile(nom_table, colname, q=0.25)
    q50 = quantile(nom_table, colname, q=0.5)
    q75 = quantile(nom_table, colname, q=0.75)
    std = get_std(nom_table, colname)

    return f"""
        count : {count[0][0]}
        min : {min[0][0]}
        mean : {avg[0][0]}
        25% : {q25[0][0]}
        50% : {q50[0][0]}
        75% : {q75[0][0]}
        max : {max[0][0]}
    """

def quantile(nom_table, colname, q=0.5):
    """
    Retourne la valeur du quantile basé sur le nom de la table et de la colonne
    """

     #sort in ascending order with each row index
    first_step = f"select {colname},ROW_NUMBER() over (order by {colname} ) as row_id from {nom_table} WHERE {colname} <> 'None' order by {colname} asc"
    query = f"""
        with quant as ({first_step})
        select {colname} as Q{str(q * 10)} from quant where row_id = CEIL((SELECT COUNT(*) FROM quant) * {q})
    """#Q{str(q*100)} =>Q25 for q=0.25
    run_it = %sql {query}
    return run_it

def get_std(nom_table, colname):
    """
    Retourne l'écart-type d'un ensemble de valeurs dans une colonne
    """
    mean = %sql select AVG({colname}) from {nom_table}
    query = f"select SQRT((SUM(POWER({colname} - {mean[0][0]}, 2)))/count({colname})) as std from {nom_table}"
    query = %sql {query}
    return query
